#  Unit 2.4a Using Programs with Data, SQLAlchemy
> Using Programs with Data is focused on SQL and database actions.  Part A focuses on SQLAlchemy and an OOP programming style,
- toc: true
- categories: []
- type: ap
- week: 26

<span style="color:blue">**All the writing in this blog that is BLUE is my notes (besides some links)**</span>.

# Database and SQLAlchemy
> In this blog we will explore using programs with data, focused on Databases.  We will use SQLite Database to learn more about using Programs with Data.

- College Board talks about ideas like 
    - <mark>Program Usage.</mark> "iterative and interactive way when processing information"
    - <mark>Managing Data.</mark>  "classifying data are part of the process in using programs", "data files in a Table"
    - <mark>Insight.</mark> "insight and knowledge can be obtained from ...  digitally represented information"
    - <mark>Filter systems.</mark> 'tools for finding information and recognizing patterns"
    - <mark>Application.</mark> "the preserve has two databases", "an employee wants to count the number of book"

- PBL, Databases, Iterative/OOP
    - <mark>Iterative.</mark> Refers to a sequence of instructions or code being repeated until a specific end result is achieved
    - <mark>OOP.</mark> A computer programming model that organizes software design around data, or objects, rather than functions and logic
    - <mark>SQL.</mark> Structured Query Language, abbreviated as SQL, is a language used in programming, managing, and structuring data 


## Imports and Flask Objects
> Defines and key object creations

Comment on where you have observed these working?

- <span style="color:blue"> 1. Flask app object: Flask app object has been used in trimester two's flask backend server for the final project.</span>

- <span style="color:blue"> 2. SQLAlchemy object: SQLAlchemy object has been used in trimester two's flask backend server for the final project.</span>


In [23]:
"""
These imports define the key objects
"""

from flask import Flask
from flask_sqlalchemy import SQLAlchemy

"""
These object and definitions are used throughout the Jupyter Notebook.
"""

# Setup of key Flask object (app)
app = Flask(__name__)
# Setup SQLAlchemy object and properties for the database (db)
database = 'sqlite:///sqlite.db'  # path and filename of database
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False
app.config['SQLALCHEMY_DATABASE_URI'] = database
app.config['SECRET_KEY'] = 'SECRET_KEY'
db = SQLAlchemy()


# This belongs in place where it runs once per project
db.init_app(app)


## Model Definition
> Define columns, initialization, and CRUD methods for users table in sqlite.db

Items



- <span style="color:blue"> class User purpose: class User is used to define attributes of data in a table and their properties.</span>

- <span style="color:blue"> db.Model inheritance: db.Model creates the layout that stores the data and determines how the data will be used.</span>

- <span style="color:blue"> _init_ method: _init_ methods are used to define the attributes of a table that will be created.</span>

- <span style="color:blue"> @property: A getter method that is used to show data(read).</span>

- <span style="color:blue"> @setter: A setter method that is used to change data(create).</span>

- <span style="color:blue"> additional methods: Update method and Delete method.</span>

In [24]:
""" database dependencies to support sqlite examples """
import datetime
from datetime import datetime
import json

from sqlalchemy.exc import IntegrityError
from werkzeug.security import generate_password_hash, check_password_hash


''' Tutorial: https://www.sqlalchemy.org/library.html#tutorials, try to get into a Python shell and follow along '''

# Define the User class to manage actions in the 'users' table
# -- Object Relational Mapping (ORM) is the key concept of SQLAlchemy
# -- a.) db.Model is like an inner layer of the onion in ORM
# -- b.) User represents data we want to store, something that is built on db.Model
# -- c.) SQLAlchemy ORM is layer on top of SQLAlchemy Core, then SQLAlchemy engine, SQL
class User(db.Model):
    __tablename__ = 'userspeople2'  # table name is plural, class name is singular

    # Define the User schema with "vars" from object
    id = db.Column(db.Integer, primary_key=True)
    _name = db.Column(db.String(255), unique=False, nullable=False)
    _uid = db.Column(db.String(255), unique=True, nullable=False)
    _password = db.Column(db.String(255), unique=False, nullable=False)
    _dob = db.Column(db.Date)

    # constructor of a User object, initializes the instance variables within object (self)
    def __init__(self, name, uid, password="123qwerty", dob=datetime.today()):
        self._name = name    # variables with self prefix become part of the object, 
        self._uid = uid
        self.set_password(password)
        if isinstance(dob, str):  # not a date type     
            dob = date=datetime.today()
        self._dob = dob

    # a name getter method, extracts name from object
    @property
    def name(self):
        return self._name
    
    # a setter function, allows name to be updated after initial object creation
    @name.setter
    def name(self, name):
        self._name = name
    
    # a getter method, extracts email from object
    @property
    def uid(self):
        return self._uid
    
    # a setter function, allows name to be updated after initial object creation
    @uid.setter
    def uid(self, uid):
        self._uid = uid
        
    # check if uid parameter matches user id in object, return boolean
    def is_uid(self, uid):
        return self._uid == uid
    
    @property
    def password(self):
        return self._password[0:10] + "..." # because of security only show 1st characters

    # update password, this is conventional setter
    def set_password(self, password):
        """Create a hashed password."""
        self._password = generate_password_hash(password, method='sha256')

    # check password parameter versus stored/encrypted password
    def is_password(self, password):
        """Check against hashed password."""
        result = check_password_hash(self._password, password)
        return result
    
    # dob property is returned as string, to avoid unfriendly outcomes
    @property
    def dob(self):
        dob_string = self._dob.strftime('%m-%d-%Y')
        return dob_string
    
    # dob should be have verification for type date
    @dob.setter
    def dob(self, dob):
        if isinstance(dob, str):  # not a date type     
            dob = date=datetime.today()
        self._dob = dob
    
    @property
    def age(self):
        today = datetime.today()
        return today.year - self._dob.year - ((today.month, today.day) < (self._dob.month, self._dob.day))
    
    # output content using str(object) in human readable form, uses getter
    # output content using json dumps, this is ready for API response
    def __str__(self):
        return json.dumps(self.read())

    # CRUD create/add a new record to the table
    # returns self or None on error
    def create(self):
        try:
            # creates a person object from User(db.Model) class, passes initializers
            db.session.add(self)  # add prepares to persist person object to Users table
            db.session.commit()  # SqlAlchemy "unit of work pattern" requires a manual commit
            return self
        except IntegrityError:
            db.session.remove()
            return None

    # CRUD read converts self to dictionary
    # returns dictionary
    def read(self):
        return {
            "id": self.id,
            "name": self.name,
            "uid": self.uid,
            "dob": self.dob,
            "age": self.age,
        }

    # CRUD update: updates user name, password, phone
    # returns self
    def update(self, name="", uid="", password=""):
        """only updates values with length"""
        if len(name) > 0:
            self.name = name
        if len(uid) > 0:
            self.uid = uid
        if len(password) > 0:
            self.set_password(password)
        db.session.commit()
        return self

    # CRUD delete: remove self
    # None
    def delete(self):
        db.session.delete(self)
        db.session.commit()
        return None
    

## Initial Data
> Uses SQLALchemy db.create_all() to initialize rows into sqlite.db

Comment on how these work?



- <span style="color:blue"> 1. Create All Tables from db Object: The command used to create All Tables is db.create_all with db being used as the database</span>

- <span style="color:blue"> 2. User Object Constructors: User object constructors work by defining all the variables in a user by defining them to the dataset.</span>

- <span style="color:blue"> 3. Try / Except: Try/expect first tries to do something in the try section but if that section of code results in an error, the expect section is ran.</span>


In [25]:
"""Database Creation and Testing """


# Builds working data for testing
def initUsers():
    with app.app_context():
        """Create database and tables"""
        db.create_all()
        """Tester data for table"""
        u1 = User(name='Thomas Edison', uid='toby', password='123toby', dob=datetime(1847, 2, 11))
        u2 = User(name='Nikola Tesla', uid='niko', password='123niko')
        u3 = User(name='Alexander Graham Bell', uid='lex', password='123lex')
        u4 = User(name='Eli Whitney', uid='whit', password='123whit')
        u5 = User(name='Indiana Jones', uid='indi', dob=datetime(1920, 10, 21))
        u6 = User(name='Marion Ravenwood', uid='raven', dob=datetime(1921, 10, 21))


        users = [u1, u2, u3, u4, u5, u6]

        """Builds sample user/note(s) data"""
        for user in users:
            try:
                '''add user to table'''
                object = user.create()
                print(f"Created new uid {object.uid}")
            except:  # error raised if object nit created
                '''fails with bad or duplicate data'''
                print(f"Records exist uid {user.uid}, or error.")
                
initUsers()

Records exist uid toby, or error.
Records exist uid niko, or error.
Records exist uid lex, or error.
Records exist uid whit, or error.
Records exist uid indi, or error.
Records exist uid raven, or error.


## Check for given Credentials in users table in sqlite.db
> Use of ORM Query object and custom methods to identify user to credentials uid and password

Purpose of the following



- <span style="color:blue"> 1. User.query.filter_by: A line of code used to go get the data from a dataset and sort it by what whatever comes after the statement.</span>

- <span style="color:blue"> 2. user.password: used to check the password, get the password from a user, or used to update a password</span>

In [26]:
# SQLAlchemy extracts single user from database matching User ID
def find_by_uid(uid):
    with app.app_context():
        user = User.query.filter_by(_uid=uid).first()
    return user # returns user object

# Check credentials by finding user and verify password
def check_credentials(uid, password):
    # query email and return user record
    user = find_by_uid(uid)
    if user == None:
        return False
    if (user.is_password(password)):
        return True
    return False
        
#check_credentials("indi", "123qwerty")

## Create a new User in table in Sqlite.db
> Uses SQLALchemy and custom user.create() method to add row.

Purpose of the following



- <span style="color:blue"> 1. user.find_by_uid() and try/except: user.find_by_uid is used to set the result of the function "find_by_uid" to the new user variable. "find_by_uid" returns user and it set to the user variable that it outside the function. Try/expect first tries to do something in the try section but if that section of code results in an error, the expect section is ran.</span>

- <span style="color:blue"> 2. user = User(...): user = User(...) sets user to have the attributes of an object created by the class User.</span>

- <span style="color:blue"> 3. user.dob and try/except: user.dob is used in order to be set to a date of one of the user's birthday. Try/expect first tries to do something in the try section but if that section of code results in an error, the expect section is ran. In this case, the expect section runs if a date of birth is not a real date or if it is none.</span>

- <span style="color:blue"> 4. user.create() and try/except: used to add a new user id and all the other attributes of user. Try/expect first tries to do something in the try section but if that section of code results in an error, the expect section is ran. In this case, the expect section runs if the program doeesn't know what the uid is.</span>

In [ ]:
# Inputs, Try/Except, and SQLAlchemy work together to build a valid database object
def create():
    # optimize user time to see if uid exists
    uid = input("Enter your user id:")
    user = find_by_uid(uid)
    try:
        print("Found\n", user.read())
        return
    except:
        pass # keep going
    
    # request value that ensure creating valid object
    name = input("Enter your name:")
    password = input("Enter your password")
    
    # Initialize User object before date
    user = User(name=name, 
                uid=uid, 
                password=password
                )
    
    # create user.dob, fail with today as dob
    dob = input("Enter your date of birth 'YYYY-MM-DD'")
    try:
        user.dob = datetime.strptime(dob, '%Y-%m-%d').date()
    except ValueError:
        user.dob = datetime.today()
        print(f"Invalid date {dob} require YYYY-mm-dd, date defaulted to {user.dob}")
           
    # write object to database
    with app.app_context():
        try:
            object = user.create()
            print("Created\n", object.read())
        except:  # error raised if object not created
            print("Unknown error uid {uid}")
        
create()

## Reading users table in sqlite.db
> Uses SQLALchemy query.all method to read data

Purpose of the following



- <span style="color:blue"> 1. User.query.all: User.query.all is set to table as a way to filter the users in the table by id.</span>

- <span style="color:blue"> 2. json_ready assignment: json_ready is a variable set to read all the users in the table so that it will be printed out to show the data in the return statment.</span>

In [14]:

# SQLAlchemy extracts all users from database, turns each user into JSON
def read():
    with app.app_context():
        table = User.query.all()
    json_ready = [user.read() for user in table] # each user adds user.read() to list
    return json_ready

read()

[{'id': 1,
  'name': 'Thomas Edison',
  'uid': 'toby',
  'dob': '02-11-1847',
  'age': 176},
 {'id': 2,
  'name': 'Nikola Tesla',
  'uid': 'niko',
  'dob': '03-15-2023',
  'age': 0},
 {'id': 3,
  'name': 'Alexander Graham Bell',
  'uid': 'lex',
  'dob': '03-15-2023',
  'age': 0},
 {'id': 4,
  'name': 'Eli Whitney',
  'uid': 'whit',
  'dob': '03-15-2023',
  'age': 0},
 {'id': 5,
  'name': 'Indiana Jones',
  'uid': 'indi',
  'dob': '10-21-1920',
  'age': 102},
 {'id': 6,
  'name': 'Marion Ravenwood',
  'uid': 'raven',
  'dob': '10-21-1921',
  'age': 101},
 {'id': 7, 'name': '', 'uid': '', 'dob': '03-15-2023', 'age': 0}]

# Hacks

In [27]:
import json
from sqlalchemy.exc import IntegrityError
  
class food(db.Model):
    __tablename__ = 'fooditems'

    id = db.Column(db.Integer, primary_key=True)
    _name = db.Column(db.String(255), unique=True, nullable=False)
    _points = db.Column(db.String(255), unique=False, nullable=False)
    _image = db.Column(db.String(255), unique=True, nullable=False)

    def __init__(self, name, points, image):
        self._name = name
        self._points = points
        self._image = image

    @property
    def name(self):
       return self._name
    @name.setter
    def name(self, name):
        self._name = name

    @property
    def points(self):
       return self._points
    @points.setter
    def points(self, points):
       self._points = points
       
    @property
    def image(self):
       return self._image
    @image.setter
    def image(self, image):
       self._image = image

    def __str__(self):
       return json.dumps(self.read())
   
    def create(self):
        try:
            db.session.add(self)
            db.session.commit()
            return self
        except IntegrityError:
            db.session.remove()
            return None
    
    def read(self):
       return {
           "id": self.id,
           "name" : self.name,
           "points" : self.points,
           "image" : self.image
       }

        
    def update(self, name="", points="", image=""):
       """only updates values with length"""
       if len(name) > 0:
           self.name = name
       if len(points) > 0:
           self.restaurant = points
       if len(image) > 0:
           self.protein = image
       db.session.commit()
       return self
   
    def delete(self):
        db.session.delete(self)
        db.session.commit()
        return None
    
def initfooditem():
    with app.app_context():
        db.create_all()
        u1 = food(name='Burger', points='10', image='https://png.pngtree.com/png-vector/20190130/ourlarge/pngtree-cute-minimalist-creative-cartoon-hamburger-png-image_611163.jpg')
        u2 = food(name= 'Fries', points='20', image='https://thumbs.dreamstime.com/b/french-fries-cartoon-clipart-red-paper-box-carton-121897301.jpg')

        fooditems = [u1, u2]

        for h in fooditems:
            try:
                h.create()
            except IntegrityError:
                db.session.remove()
                print(f"Records exist, duplicate, or error: {food.id}")
                
initfooditem()

### Update

In [ ]:
def locateitem(name):
    with app.app_context():
        item = food.query.filter_by(name=name).first()
    return item

def update():
    name = input("Enter name of item")
    item = locateitem(name)
    try:
        print(item.read())
        pass
    except:
        return("Unknown Name")
    
    points = input("Enter new points")
    image = input("Enter link of image")
    
    with app.app_context():
        try:
            updateditem = item.update(name, points, image)
            print(updateditem.read())
        except:  # error raised if object not created
            print("Unknown error uid {uid}")
        
update()

### Delete

In [ ]:
def locateitem(name):
    with app.app_context():
        item = food.query.filter_by(name=name).first()
    return item

def delete():
    name = input("Enter name of item")
    item = locateitem(name)
    try:
        print(item.read())
        pass
    except:
        return("Unknown Name")
    
    with app.app_context():
        try:
            item.delete()
            print("item has been deleted")
        except:
            print("Unknown error uid {uid}")

delete()